In [24]:
from models.arcs import load_alt_model_a, load_alt_model_b, load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, map_value_to_index, init_sequences_a, init_sequences_b
from utilities.visualizers import export_results

from tensorflow.keras.losses import CategoricalCrossentropy as cce_loss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, CategoricalCrossentropy as cce_metric

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
corpus = load_file('./data/notes.txt')

In [26]:
corpus[:500]

'A simple idea “What is the meaning of life?”\nI asked as I learned through the works of Camus? \nOne step down, I felt a yearning of meaning in this world.\nIn this yearning I stumbled upon eastern philosophy;\nIkigai as the Japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nFollowed then another idea, a leap of faith as Kierkegaard would call it, yet I had no idea this was his idea. \nCalm followed after the storm, and then I took another'

In [27]:
len(corpus)

226750

In [28]:
chars = sorted(list(set(corpus)))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…']

# Preprocessing corpus
* replace quotation marks like this '“'/'”' with this instead '"'
* replace single quotation marks like this '‘'/'’' with ''' instead
* replace this hyphen '–' with this hyphen '—'
* lowercase all words (for now)
* replace 3 consecutive '.' with  '…' instead

In [29]:
corpus = preprocess(corpus)
corpus[:500]

'a simple idea "what is the meaning of life?"\ni asked as i learned through the works of camus? \none step down, i felt a yearning of meaning in this world.\nin this yearning i stumbled upon eastern philosophy;\nikigai as the japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nfollowed then another idea, a leap of faith as kierkegaard would call it, yet i had no idea this was his idea. \ncalm followed after the storm, and then i took another'

In [30]:
len(corpus)

226961

In [31]:
chars = sorted(list(set(corpus)))
# chars = ['[UNK]'] + chars
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

# Creating mapper from a unique character to its respective index

In [32]:
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [33]:
char_to_idx

In [34]:
idx_to_char

In [35]:
n_unique = len(char_to_idx.get_vocabulary())
n_unique

57

In [36]:
n_time_steps = 100
X, Y = init_sequences_a(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(226861, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [ 2, 43, 33, ...,  2, 43, 44],
       [43, 33, 37, ..., 43, 44, 29],
       ...,
       [39, 42,  2, ..., 29, 36, 33],
       [42,  2, 44, ..., 36, 33, 29],
       [ 2, 44, 42, ..., 33, 29, 30]], dtype=int64)>

In [37]:
X.shape

TensorShape([226861, 100])

In [38]:
Y

<tf.Tensor: shape=(226861,), dtype=int64, numpy=array([44, 29, 40, ..., 29, 30, 11], dtype=int64)>

# convert Y data's indeces to their one hot vector representation

In [39]:
Y = tf.one_hot(Y, depth=n_unique)
Y

<tf.Tensor: shape=(226861, 57), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [40]:
Y[3]

<tf.Tensor: shape=(57,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>

In [41]:
# number of examples
len(X)

226861

In [42]:
len(X[-1])

100

# Instantiate generative model A with set architecture

In [43]:
emb_dim = 64
n_a = 32

In [44]:
model = load_alt_model_a(n_unique=n_unique, T_x=n_time_steps, emb_dim=emb_dim, n_a=n_a)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           3648      
                                                                 
 lstm (LSTM)                 (None, 100, 32)           12416     
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 57)                1881      
                                                                 
 batch_normalization (Batch  (None, 57)                228       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 57)                0         
                                                        

# Provide loss, optimizer, and metrics for both alternative models A and B

In [45]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
loss = cce_loss()
metrics = [CategoricalAccuracy(), cce_metric()]

# model.compile(loss=loss, optimizer=opt, metrics=metrics)

# Train alternative model A and checkpoint weights

In [46]:
# weights_path = "./weights/weights-improvement-{epoch:02d}-{categorical_accuracy:.4f}.hdf5"
# checkpoint = ModelCheckpoint(weights_path, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

In [47]:
# history = model.fit(X, Y, epochs=20, batch_size=2048, callbacks=callbacks_list)

In [48]:
# export_results(history, ['loss'], image_only=False)
# export_results(history, ['categorical_accuracy'], image_only=False)

# Preprocessing for alternative model B

In [50]:
X, Y = init_sequences_b(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [29, 40,  2, ..., 36, 39, 43],
       [40, 32, 49, ..., 25, 44,  2],
       ...,
       [39,  2, 30, ..., 32, 33, 43],
       [44, 32, 29, ...,  2, 26, 42],
       [25, 35, 43, ...,  0,  0,  0]], dtype=int64)>

In [51]:
len(X)

2248

In [52]:
len(X[-2])

100

In [53]:
len(X[-1])

100

In [54]:
len(Y[-1])

100

In [55]:
X[-1]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([25, 35, 43,  2, 37, 49,  2, 26, 29, 36, 33, 29, 30,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int64)>

# Load a saved model
* see arhictecure
* see if prediction will work properly on dummy data

In [56]:
saved_model = load_model('./saved/models/test_model_b.h5')

In [57]:
saved_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 X (InputLayer)              [(None, 50)]                 0         []                            
                                                                                                  
 character-lookup (Embeddin  (None, 50, 32)               832       ['X[0][0]']                   
 g)                                                                                               
                                                                                                  
 tf.__operators__.getitem (  (None, 32)                   0         ['character-lookup[0][0]']    
 SlicingOpLambda)                                                                                 
                                                                                              

#### Recall that our model needs 3 inputs, X, the hidden state, and the cell state. Because we are generating novel sequences using our trained model we pass in a $(1, 100)$ input where it represents the shape $(m, T_x)$, moreover our hidden and cell states remain the same in terms of their shape which is $(m, n_a)$ but only now it would be $(1, n_a)$ since we are passing only one input example to our model

In [58]:
sample_input = tf.random.uniform(shape=(1, 50), minval=0, maxval=25, dtype=tf.int32)

sample_h = tf.zeros(shape=(1, 128))
sample_c = tf.zeros(shape=(1, 128))

#### use the model to predict an output Y which we know will be of shape $(T_y, m, n_{unique})$ or in this case since we only inputted one example $(T_y, 1, 26)$

In [59]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 37s 37s/step


[array([[ -6.313008  ,   2.8073983 ,  -0.17492348, -14.137193  ,
          -5.549599  ,  -8.058135  ,  -5.706407  ,   8.853065  ,
          -2.861485  ,   5.6493835 ,   1.6117928 ,  -8.542794  ,
           0.38485545,  -2.8477347 , -11.210341  ,  -4.099345  ,
          -3.6742063 , -14.114946  ,   4.5422697 ,   4.429382  ,
           6.9211473 ,   5.0116673 , -10.144202  , -12.499184  ,
           4.2832737 ,  14.38207   ]], dtype=float32),
 array([[ -6.9975514 ,   3.3441062 ,   0.14841434, -13.291225  ,
          -5.7730727 ,  -8.110076  ,  -5.947029  ,   9.130415  ,
          -2.852258  ,   6.4949865 ,   1.8702359 ,  -9.796644  ,
           0.85069096,  -3.157543  , -10.836209  ,  -4.5436707 ,
          -3.8378074 , -13.324709  ,   4.293727  ,   3.9615073 ,
           7.0711975 ,   5.0024776 , -10.661073  , -12.077406  ,
           3.6384501 ,  15.284601  ]], dtype=float32),
 array([[ -6.680928  ,   3.850476  ,  -0.07778865, -12.533722  ,
          -6.2335253 ,  -7.480307  ,  -5.8768

#### List all layer names
* the goal here is to extract the Embedding, LSTM, Dense, and BatchNormalization layers used in training the model which have all been instantiated once and where we can extract and use as objects in our inference model
* Once we know the layers, we can start by training our model for training, and save the best model with the lowest loss value
* Access the saved models aforementioned layers and use it in the inference model

In [60]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

X
character-lookup
tf.__operators__.getitem
reshape-layer
init-hidden-state
init-cell-state
tf.__operators__.getitem_1
lstm-cell
tf.__operators__.getitem_2
tf.__operators__.getitem_3
tf.__operators__.getitem_4
tf.__operators__.getitem_5
tf.__operators__.getitem_6
tf.__operators__.getitem_7
tf.__operators__.getitem_8
tf.__operators__.getitem_9
tf.__operators__.getitem_10
tf.__operators__.getitem_11
tf.__operators__.getitem_12
tf.__operators__.getitem_13
tf.__operators__.getitem_14
tf.__operators__.getitem_15
tf.__operators__.getitem_16
tf.__operators__.getitem_17
tf.__operators__.getitem_18
tf.__operators__.getitem_19
tf.__operators__.getitem_20
tf.__operators__.getitem_21
tf.__operators__.getitem_22
tf.__operators__.getitem_23
tf.__operators__.getitem_24
tf.__operators__.getitem_25
tf.__operators__.getitem_26
tf.__operators__.getitem_27
tf.__operators__.getitem_28
tf.__operators__.getitem_29
tf.__operators__.getitem_30
tf.__operators__.getitem_31
tf.__operators__.getitem_32
tf.__operat

In [61]:
lstm_cell = saved_model.get_layer('lstm-cell')
embedding_layer = saved_model.get_layer('character-lookup')
dense_layer = saved_model.get_layer('dense-layer')
norm_layer = saved_model.get_layer('norm-layer')

In [62]:
lstm_cell.get_weights()

[array([[ 0.12204831,  0.13164113,  0.09484749, ..., -0.02121187,
         -0.01332705, -0.01091615],
        [ 0.05405626, -0.04563445, -0.04914114, ..., -0.02561642,
         -0.22203676,  0.19439848],
        [ 0.01579465,  0.019968  , -0.13640566, ..., -0.00085213,
         -0.05333597, -0.23642084],
        ...,
        [ 0.10468645, -0.06170968,  0.09743046, ..., -0.0839068 ,
         -0.04235022,  0.01842632],
        [ 0.00684932,  0.12458528,  0.06192198, ..., -0.05839727,
          0.01287928,  0.15863056],
        [-0.13451274,  0.1960856 , -0.08487105, ...,  0.01150034,
          0.13469619, -0.03600522]], dtype=float32),
 array([[-0.11174878, -0.08285551, -0.03651781, ..., -0.00822446,
         -0.12738374, -0.19310722],
        [-0.01256288,  0.09092997,  0.02405682, ...,  0.06650868,
          0.10675476,  0.08209385],
        [-0.13109353, -0.00995022, -0.05971202, ..., -0.00104832,
          0.0220722 , -0.07407735],
        ...,
        [ 0.0038699 ,  0.05910082,  0.0

In [63]:
embedding_layer.get_weights()

[array([[-4.47490662e-02,  5.15247555e-03,  7.07732216e-02,
         -4.65096459e-02,  3.80752073e-03, -4.67416085e-02,
          9.56830457e-02, -8.76945406e-02,  3.52438502e-02,
         -4.58007231e-02,  4.11500260e-02, -6.97602257e-02,
          9.77647603e-02,  5.97999655e-02, -1.69803035e-02,
          7.25868717e-03,  1.46029234e-01, -6.13973886e-02,
         -1.05658107e-01,  8.40007607e-03,  2.77843606e-02,
          3.72205228e-02,  1.03594579e-01,  1.98535621e-03,
         -9.00326744e-02, -7.22664595e-02, -1.00718038e-02,
          7.27922563e-03, -1.15613684e-01, -9.15359557e-02,
          5.08964621e-02, -7.48558268e-02],
        [-4.15506773e-02, -8.57126042e-02, -4.66780141e-02,
          9.90973562e-02, -3.37406620e-02, -4.52356152e-02,
          7.48232007e-02,  4.93014380e-02, -2.63393819e-02,
         -8.24911073e-02,  6.74934834e-02,  9.07506701e-03,
          5.91191314e-02,  6.47601485e-02, -6.97327182e-02,
         -5.99812642e-02, -2.15695854e-02, -4.33918238e-

In [64]:
dense_layer.get_weights()

[array([[ 0.11579768,  0.00031461,  0.17468622, ..., -0.07475954,
          0.06511027, -0.17205667],
        [-0.09899703, -0.04995552,  0.08411463, ..., -0.0313869 ,
          0.066606  , -0.19079526],
        [ 0.11255792, -0.01575104,  0.06275032, ..., -0.15834048,
          0.06150521, -0.01086924],
        ...,
        [-0.10740884,  0.10040522,  0.14096043, ..., -0.17221639,
         -0.1450935 , -0.24378334],
        [-0.05766907, -0.07602404,  0.15259291, ...,  0.0617708 ,
         -0.11626799, -0.06028632],
        [ 0.18549454,  0.22468509,  0.17586151, ..., -0.06697255,
         -0.1414097 ,  0.11651406]], dtype=float32),
 array([ 0.03434021,  0.02106909,  0.01311352, -0.03171585,  0.00169511,
         0.0176196 , -0.02090266,  0.01995847, -0.01448148, -0.00560184,
        -0.01579482,  0.0115351 , -0.0244345 ,  0.0134882 , -0.00470636,
         0.0064377 , -0.01442294,  0.00589113,  0.00067807,  0.03149892,
         0.04390266,  0.05082439,  0.00226096, -0.01831539,  0.019

In [65]:
norm_layer.get_weights()

[array([1.2647594, 1.2878861, 1.2234375, 1.279036 , 1.2648399, 1.2147951,
        1.3110735, 1.3156829, 1.3030491, 1.241658 , 1.2250794, 1.2197847,
        1.3066726, 1.3045917, 1.2822499, 1.2760696, 1.2660922, 1.2703315,
        1.2315006, 1.2787199, 1.2804294, 1.2873979, 1.3154113, 1.2513084,
        1.266361 , 1.295395 ], dtype=float32),
 array([-0.01555031,  0.03749332, -0.06076985,  0.09425076, -0.07044636,
        -0.08798521,  0.02167816, -0.04626423, -0.00883236,  0.04325766,
         0.04503202,  0.02115029, -0.0666273 ,  0.05720158,  0.08667522,
        -0.09022159,  0.028329  , -0.02400754, -0.00122527,  0.03748413,
         0.10020015, -0.00495446, -0.02937842,  0.0124602 , -0.01619693,
        -0.16243827], dtype=float32),
 array([ 0.29391238, -0.09568492,  0.01212041,  0.58601844,  0.21453215,
         0.4058981 ,  0.20710112, -0.33873215,  0.09735097, -0.1965334 ,
        -0.03058777,  0.34685305, -0.02180245,  0.11473003,  0.4283333 ,
         0.15675308,  0.19594799,  

# Load saved weights

In [66]:
saved_model = GenPhiloText()
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 26), dtype=float32, numpy=
 array([[-4.1029784e-03,  1.9361688e-03,  3.7947563e-03, -3.7514761e-03,
          5.1149819e-03,  8.6088735e-04,  2.8415378e-03,  6.0625636e-04,
          8.9139369e-04,  5.8273750e-04,  1.0141873e-02, -8.7775812e-03,
          1.8640703e-03,  7.4600175e-05,  1.8226010e-03, -2.2818158e-04,
          3.4335235e-03,  1.2190532e-03, -3.4593611e-03, -1.1348226e-03,
         -1.2078728e-03,  7.2401459e-04,  4.6780277e-03,  5.5265916e-03,
         -2.7640502e-03, -6.1020418e-03]], dtype=float32)>,
 <tf.Tensor: shape=(1, 26), dtype=float32, numpy=
 array([[-0.00317658,  0.00439597,  0.00699914,  0.00179708,  0.00580848,
          0.00030138,  0.00239825, -0.00022031, -0.00327104,  0.00280132,
          0.00876093, -0.00747736,  0.00283056,  0.0015822 , -0.00683366,
          0.00181472,  0.00507702,  0.00238358, -0.00837567, -0.00395998,
          0.00392438,  0.00032077,  0.00318594,  0.00404195, -0.00252505,
         -0.0052076 ]], dtype=fl

In [74]:
saved_model.summary()

Model: "gen_philo_text"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  832       
                                                                 
 lstm-cell (LSTM)            multiple                  82432     
                                                                 
 dense-layer (Dense)         multiple                  3354      
                                                                 
 activation_1 (Activation)   multiple                  0 (unused)
                                                                 
 reshape-layer (Reshape)     multiple                  0         
                                                                 
 norm-layer (BatchNormaliza  multiple                  104       
 tion)                                                           
                                                    

In [67]:
saved_model.load_weights(filepath='./saved/weights/test_model_gen_philo_text.h5')

In [68]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 23s 23s/step


[array([[ -5.3876476 ,   2.0375483 ,  -3.962681  ,  -8.484262  ,
         -20.599207  ,  17.121416  ,  14.863677  ,  -7.8075514 ,
           2.3156605 ,  -2.5487876 ,  -8.196767  ,   3.0104446 ,
           3.326107  ,   6.9449663 ,  -3.9281607 ,  15.143926  ,
          -5.305874  ,   9.630028  ,   1.2624763 ,   7.6678824 ,
         -10.173723  ,  -0.05602711,  10.641521  ,   4.4106035 ,
           1.1457984 ,  -9.766292  ]], dtype=float32),
 array([[ -4.753745  ,   1.6759069 ,  -3.4237309 ,  -7.9759483 ,
         -21.395998  ,  17.181343  ,  15.023958  ,  -9.05051   ,
           1.2300327 ,  -1.8918608 ,  -7.402089  ,   2.925922  ,
           3.9526432 ,   7.283831  ,  -4.680322  ,  14.827131  ,
          -4.976079  ,   9.423865  ,   1.0860375 ,   8.095992  ,
         -10.340733  ,   0.62431926,  10.502456  ,   3.6533494 ,
           1.3786154 ,  -9.837263  ]], dtype=float32),
 array([[ -3.6113763 ,   0.59717643,  -3.941507  ,  -8.289752  ,
         -21.984007  ,  17.863697  ,  14.9786

In [69]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer
activation_1
reshape-layer
norm-layer


In [70]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
char_emb_layer.get_weights()

[array([[ 2.08263155e-02, -6.06079847e-02, -6.50052801e-02,
         -1.09132588e-01,  4.88749752e-03,  1.32442545e-02,
          1.47926748e-01, -1.63190570e-02,  1.37823090e-01,
         -4.28359620e-02,  6.71227053e-02, -2.20562946e-02,
          2.40952149e-03, -8.40303525e-02, -6.76146150e-02,
         -3.18596698e-02,  5.84085509e-02, -1.69190783e-02,
         -4.44295891e-02, -7.09246797e-03, -1.08491629e-01,
         -3.55349816e-02, -3.92042380e-03,  2.60478148e-04,
          6.04924150e-02, -4.04502116e-02, -2.76143644e-02,
         -7.61242136e-02, -2.05826424e-02, -6.42829984e-02,
         -7.74821192e-02,  2.84780841e-02],
        [ 5.78687042e-02, -1.05880588e-01,  7.33178295e-03,
         -3.28982361e-02, -5.96830482e-03,  2.71778740e-02,
         -7.41622671e-02,  2.85282321e-02,  1.48376957e-01,
         -1.14882410e-01, -1.33319184e-01,  3.36326286e-02,
         -4.28270511e-02,  3.79670188e-02,  1.57988057e-01,
         -9.59969610e-02,  7.04875886e-02,  3.76116112e-

In [71]:
lstm_cell = saved_model.get_layer('lstm-cell')
lstm_cell.get_weights()

[array([[ 0.00956698, -0.12273197,  0.01113207, ..., -0.07912294,
         -0.11982283,  0.11448838],
        [-0.10146036,  0.00157733,  0.01891055, ...,  0.07731269,
          0.08626043, -0.11665835],
        [-0.05595766,  0.04494354,  0.07241275, ..., -0.1271429 ,
          0.03540231,  0.23710693],
        ...,
        [-0.13439973, -0.04190993,  0.09469479, ..., -0.02175266,
         -0.04648986, -0.05066203],
        [-0.11396432,  0.02203524, -0.05030974, ..., -0.05100011,
          0.17830136, -0.11556329],
        [-0.12456088, -0.04636979,  0.16456527, ...,  0.05314025,
         -0.04629907, -0.01683245]], dtype=float32),
 array([[ 0.02838516,  0.0058615 , -0.08817896, ..., -0.04304688,
         -0.01959699, -0.12579927],
        [-0.15300496, -0.08215949, -0.02372912, ..., -0.06801607,
          0.09040465,  0.03870963],
        [-0.10514799,  0.00653865,  0.04740473, ...,  0.07969011,
          0.05814108, -0.00418376],
        ...,
        [ 0.06954969,  0.04687146,  0.0

In [72]:
dense_layer = saved_model.get_layer('dense-layer')
dense_layer.get_weights()

[array([[ 0.16945018, -0.09243356, -0.10993639, ..., -0.09831604,
          0.07301178,  0.04433766],
        [-0.01657312, -0.18560818, -0.00872533, ..., -0.22604862,
         -0.12873808,  0.03368679],
        [ 0.17025535, -0.23003832, -0.05234543, ...,  0.05310398,
         -0.19798298, -0.16506684],
        ...,
        [ 0.03735596, -0.09462428,  0.20676959, ...,  0.13470371,
         -0.17413   ,  0.16865395],
        [ 0.2062815 , -0.05883304, -0.03412116, ...,  0.03719562,
          0.11921233,  0.17117457],
        [-0.16814892, -0.24370237,  0.19757469, ..., -0.0072566 ,
          0.09939792,  0.23812008]], dtype=float32),
 array([-0.00547797,  0.01815883, -0.00486371,  0.00975229, -0.02873891,
        -0.03185334, -0.05412811,  0.01632123,  0.00373298, -0.02533336,
        -0.00024047,  0.01026723, -0.05447325,  0.0060126 , -0.02730132,
        -0.01770192, -0.02216223,  0.01513467,  0.02323797, -0.02374019,
         0.00995885,  0.01609473,  0.03482916, -0.00939293, -0.011

In [73]:
norm_layer = saved_model.get_layer('norm-layer')
norm_layer.get_weights()

[array([1.2745422, 1.2703699, 1.307507 , 1.2942533, 1.3357022, 1.269279 ,
        1.3347929, 1.2673551, 1.2831123, 1.2378561, 1.235446 , 1.2755878,
        1.1937706, 1.2748057, 1.2799615, 1.3301457, 1.2492634, 1.2569413,
        1.2363508, 1.2792348, 1.2740556, 1.2602367, 1.1932645, 1.2268801,
        1.1938958, 1.3167897], dtype=float32),
 array([ 0.07710075, -0.12818894, -0.04638201,  0.02649134,  0.04657882,
         0.03658828, -0.07666691,  0.222898  , -0.0195388 , -0.13060895,
         0.0230279 , -0.03643304, -0.07565083,  0.03683848, -0.0922796 ,
         0.04156287, -0.01071801,  0.00435686,  0.01063607,  0.02721214,
        -0.04486416, -0.09693124,  0.11885186, -0.04337334,  0.01561862,
        -0.04821859], dtype=float32),
 array([ 0.25203452, -0.08833419,  0.15455322,  0.34652093,  0.73427033,
        -0.73572886, -0.6151602 ,  0.32354176, -0.08980789,  0.08992064,
         0.38247526, -0.09837823, -0.19846761, -0.25783747,  0.10717244,
        -0.6353861 ,  0.23269658, -

In [84]:
load_inf_model(char_emb_layer, lstm_cell, dense_layer, norm_layer, char_to_idx)

26
128
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]], shape=(1, 26), dtype=float32)
